In [ ]:
from spacecraft.satellite import SatelliteImplementation
from setup.initial_settings import SimulationSetupReader
from setup.two_line_element import TwoLineElementReader
from visualziations.visualizations import plot_orbit, plot_lla, plot_position, plot_magnetic_field_eci, plot_magnetic_field_sbf, plot_angular_velocity, plot_euler_angles
from spacecraft.sensors import MagnetometerImplementation, SunsensorImplementation, SensorFusionImplementation
import core.transformations as tr
import core.utilities as ut
from pathlib import Path

setup = SimulationSetupReader(Path('setup/initial_settings.json'))
tle = TwoLineElementReader(Path('setup/tle'))
magnetometer = MagnetometerImplementation(noise=True)
sunsensor = SunsensorImplementation(noise=True)
sensor_fusion = SensorFusionImplementation(['triad', 'quest', 'ekf'], tr.euler_xyz_to_quaternion(setup.euler_angles))

satellite = SatelliteImplementation(setup, tle, magnetometer, sunsensor, sensor_fusion)

state_vector = ut.initialize_state_vector(satellite)
for x in range(1, 10000, 1):
    print(f"Updating iteration {x}")
    satellite.update_iteration(x)
    # satellite.apply_rotation()  # applying rotation is required for triad and quest

    mag_field_sbf, mag_field_eci = satellite.magnetic_field
    sun_vector_sbf, sun_vector_eci = satellite.sun_vector
    # print(f"Reference quaternion: {q_ref}"

    # satellite.apply_triad(
    #     ut.normalize(mag_field_eci), ut.normalize(sun_vector_eci), 
    #     ut.normalize(mag_field_sbf), ut.normalize(sun_vector_sbf)
    # )
    # q_triad = satellite.quaternion.copy()
    # print(f"Quaternion after TRIAD: {q_triad}")
    # satellite.apply_quest(
    #     [ut.normalize(mag_field_sbf), ut.normalize(sun_vector_sbf)],
    #     [ut.normalize(mag_field_eci), ut.normalize(sun_vector_eci)]
    # )
    # q_quest = satellite.quaternion.copy()
    # print(f"Quaternion after QUEST: {q_quest}")

    # ekf automatically updates the quaternion by the obtained rotation
    satellite.apply_ekf(
        [ut.normalize(mag_field_sbf), ut.normalize(sun_vector_sbf)],
        [ut.normalize(mag_field_eci), ut.normalize(sun_vector_eci)]
    )
    q_ekf = satellite.quaternion 

    satellite.apply_detumbling('b_dot')
    # print(f"Quaternion after EKF: {q_ekf}")
    state_vector = ut.update_state_vector(satellite, state_vector)


print(state_vector)
plot_orbit(state_vector, setup)
plot_position(state_vector)
plot_lla(state_vector)
plot_magnetic_field_eci(state_vector)
plot_magnetic_field_sbf(state_vector) 
plot_angular_velocity(state_vector)
plot_euler_angles(state_vector)


In [3]:
import setup.transformations as tr

euler = [50, 50, 50]

quat = tr.euler_xyz_to_quaternion(euler[0], euler[1], euler[2])
print(f"Euler angles: {quat}")
new_euler = tr.quaternion_to_euler_xyz(quat)
print(f"Converted to quaternion: {new_euler}")

Euler angles: [0.18526384 0.50900821 0.18526384 0.81991784]
Converted to quaternion: [50. 50. 50.]


In [7]:
import numpy as np

array = np.array([1.321312312, 1.53252e-6, 0.0003432])
print(np.round(array, 4))

[1.3213e+00 0.0000e+00 3.0000e-04]
